In [18]:
# Modules Import
import numpy as np
import pandas as pd

# 학습 데이터와 테스트 데이터 분리 (일반적으로 7:3 혹은 8:2)
from sklearn.model_selection import train_test_split

# 정규화
from sklearn.preprocessing import MinMaxScaler

# 이번에 사용할 데이터에는 결측치와 이상치가 없음

# 텐서플로우 - 케라스
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam # 항상 평균 이상의 성능

# 다중 분류를 위해 one-hot encoding 필요
import tensorflow as tf

print('modules imported!')

modules imported!


In [19]:
# Raw Data Loading

df = pd.read_csv('./data/bmi.csv', skiprows=3) # 4번째 줄부터 load. 1-3 라인은 정보가 적혀있음.
display(df.head(), df.shape)

# 이 데이터에는 결측치와 이상치는 존재하지 않으므로 넘어간다.
# 실제 작업시에는 확인해야한다.

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52


(20000, 3)

In [20]:
# 데이터를 학습데이터와 테스트데이터를 분리

x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height', 'weight']].values, # train_test_split(df[['height', 'weight']]) # 알아서 df를 ndarray로 변환한다
                 df['label'].values,
                 test_size=0.3) # 7:3
print(x_data_train, x_data_train.shape, end='\n\n') # (14000, 2)

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)
print(x_data_train_norm, x_data_train_norm.shape, end='\n\n\n') # 정규화됨

# one-hot 처리
# t데이터는 클래스이므로 정규화는 필요 없으나, one-hot 인코딩 필요
print(t_data_train, end='\n\n') # 현재 1차원. one-hot 처리 후 2차원

t_data_train_onehot = tf.one_hot(t_data_train, depth=3).numpy() # depth는 클래스의 수 / numpy()를 통해 tf 타입을 numpy로 변환
t_data_test_onehot = tf.one_hot(t_data_test, depth=3).numpy()
print(t_data_train_onehot, end='\n\n')
print(t_data_test_onehot, end='\n\n')

[[198  67]
 [135  41]
 [190  58]
 ...
 [149  50]
 [126  64]
 [166  61]] (14000, 2)

[[0.975      0.71111111]
 [0.1875     0.13333333]
 [0.875      0.51111111]
 ...
 [0.3625     0.33333333]
 [0.075      0.64444444]
 [0.575      0.57777778]] (14000, 2)


[0 1 0 ... 1 2 1]

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]



In [23]:
# 모델 만들기

model = Sequential()

# Layer 추가
model.add(Flatten(input_shape=(2,))) # node 2개
model.add(Dense(3, activation='softmax')) # node 3개

# Model 설정
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy', # 이항분류시 binary_crossentropy
              metrics=['accuracy']) # 모델 평가시 정확성을 accuracy로 측정하도록 지정

# Model 학습
model.fit(x_data_train_norm,
          t_data_train_onehot,
          epochs=500, # 반복 횟수
          validation_split=0.2, # 전체 학습 데이터의 일부를 평가에 사용. 평가하면서 학습 진행
          verbose=1)

Epoch 1/500
350/350 [==============================] - 1s 1ms/step - loss: 1.1659 - accuracy: 0.3229 - val_loss: 1.1543 - val_accuracy: 0.3200
Epoch 2/500
350/350 [==============================] - 0s 963us/step - loss: 1.1431 - accuracy: 0.3230 - val_loss: 1.1328 - val_accuracy: 0.3225
Epoch 3/500
350/350 [==============================] - 0s 980us/step - loss: 1.1226 - accuracy: 0.3245 - val_loss: 1.1134 - val_accuracy: 0.3254
Epoch 4/500
350/350 [==============================] - 0s 891us/step - loss: 1.1041 - accuracy: 0.3288 - val_loss: 1.0958 - val_accuracy: 0.3318
Epoch 5/500
350/350 [==============================] - 0s 883us/step - loss: 1.0871 - accuracy: 0.3349 - val_loss: 1.0797 - val_accuracy: 0.3375
Epoch 6/500
350/350 [==============================] - 0s 885us/step - loss: 1.0715 - accuracy: 0.3422 - val_loss: 1.0648 - val_accuracy: 0.3457
Epoch 7/500
350/350 [==============================] - 0s 891us/step - loss: 1.0570 - accuracy: 0.3496 - val_loss: 1.0509 - val_accu

350/350 [==============================] - 0s 878us/step - loss: 0.7140 - accuracy: 0.7929 - val_loss: 0.7165 - val_accuracy: 0.7907
Epoch 58/500
350/350 [==============================] - 0s 877us/step - loss: 0.7098 - accuracy: 0.7950 - val_loss: 0.7124 - val_accuracy: 0.7918
Epoch 59/500
350/350 [==============================] - 0s 903us/step - loss: 0.7057 - accuracy: 0.7974 - val_loss: 0.7084 - val_accuracy: 0.7929
Epoch 60/500
350/350 [==============================] - 0s 894us/step - loss: 0.7016 - accuracy: 0.7994 - val_loss: 0.7044 - val_accuracy: 0.7954
Epoch 61/500
350/350 [==============================] - 0s 917us/step - loss: 0.6976 - accuracy: 0.8021 - val_loss: 0.7004 - val_accuracy: 0.7975
Epoch 62/500
350/350 [==============================] - 0s 891us/step - loss: 0.6937 - accuracy: 0.8032 - val_loss: 0.6965 - val_accuracy: 0.8000
Epoch 63/500
350/350 [==============================] - 0s 917us/step - loss: 0.6898 - accuracy: 0.8057 - val_loss: 0.6927 - val_accuracy

350/350 [==============================] - 0s 874us/step - loss: 0.5478 - accuracy: 0.8721 - val_loss: 0.5523 - val_accuracy: 0.8654
Epoch 114/500
350/350 [==============================] - 0s 878us/step - loss: 0.5457 - accuracy: 0.8728 - val_loss: 0.5502 - val_accuracy: 0.8668
Epoch 115/500
350/350 [==============================] - 0s 883us/step - loss: 0.5437 - accuracy: 0.8737 - val_loss: 0.5482 - val_accuracy: 0.8671
Epoch 116/500
350/350 [==============================] - 0s 874us/step - loss: 0.5416 - accuracy: 0.8754 - val_loss: 0.5462 - val_accuracy: 0.8686
Epoch 117/500
350/350 [==============================] - 0s 872us/step - loss: 0.5396 - accuracy: 0.8758 - val_loss: 0.5442 - val_accuracy: 0.8696
Epoch 118/500
350/350 [==============================] - 0s 874us/step - loss: 0.5377 - accuracy: 0.8764 - val_loss: 0.5423 - val_accuracy: 0.8718
Epoch 119/500
350/350 [==============================] - 0s 860us/step - loss: 0.5357 - accuracy: 0.8783 - val_loss: 0.5403 - val_ac

Epoch 169/500
350/350 [==============================] - 0s 897us/step - loss: 0.4577 - accuracy: 0.9066 - val_loss: 0.4631 - val_accuracy: 0.9046
Epoch 170/500
350/350 [==============================] - 0s 911us/step - loss: 0.4565 - accuracy: 0.9075 - val_loss: 0.4618 - val_accuracy: 0.9050
Epoch 171/500
350/350 [==============================] - 0s 914us/step - loss: 0.4552 - accuracy: 0.9083 - val_loss: 0.4606 - val_accuracy: 0.9050
Epoch 172/500
350/350 [==============================] - 0s 934us/step - loss: 0.4540 - accuracy: 0.9081 - val_loss: 0.4594 - val_accuracy: 0.9061
Epoch 173/500
350/350 [==============================] - 0s 914us/step - loss: 0.4528 - accuracy: 0.9083 - val_loss: 0.4582 - val_accuracy: 0.9061
Epoch 174/500
350/350 [==============================] - 0s 871us/step - loss: 0.4515 - accuracy: 0.9082 - val_loss: 0.4570 - val_accuracy: 0.9061
Epoch 175/500
350/350 [==============================] - 0s 880us/step - loss: 0.4503 - accuracy: 0.9084 - val_loss: 0

Epoch 225/500
350/350 [==============================] - 0s 868us/step - loss: 0.3997 - accuracy: 0.9284 - val_loss: 0.4058 - val_accuracy: 0.9268
Epoch 226/500
350/350 [==============================] - 0s 877us/step - loss: 0.3989 - accuracy: 0.9284 - val_loss: 0.4049 - val_accuracy: 0.9261
Epoch 227/500
350/350 [==============================] - 0s 857us/step - loss: 0.3981 - accuracy: 0.9287 - val_loss: 0.4041 - val_accuracy: 0.9268
Epoch 228/500
350/350 [==============================] - 0s 894us/step - loss: 0.3972 - accuracy: 0.9291 - val_loss: 0.4033 - val_accuracy: 0.9268
Epoch 229/500
350/350 [==============================] - 0s 886us/step - loss: 0.3964 - accuracy: 0.9291 - val_loss: 0.4025 - val_accuracy: 0.9275
Epoch 230/500
350/350 [==============================] - 0s 894us/step - loss: 0.3956 - accuracy: 0.9299 - val_loss: 0.4017 - val_accuracy: 0.9275
Epoch 231/500
350/350 [==============================] - 0s 877us/step - loss: 0.3948 - accuracy: 0.9304 - val_loss: 0

Epoch 281/500
350/350 [==============================] - 0s 986us/step - loss: 0.3587 - accuracy: 0.9457 - val_loss: 0.3652 - val_accuracy: 0.9425
Epoch 282/500
350/350 [==============================] - 0s 969us/step - loss: 0.3581 - accuracy: 0.9459 - val_loss: 0.3646 - val_accuracy: 0.9425
Epoch 283/500
350/350 [==============================] - 0s 949us/step - loss: 0.3574 - accuracy: 0.9459 - val_loss: 0.3640 - val_accuracy: 0.9436
Epoch 284/500
350/350 [==============================] - 0s 920us/step - loss: 0.3568 - accuracy: 0.9461 - val_loss: 0.3634 - val_accuracy: 0.9436
Epoch 285/500
350/350 [==============================] - 0s 886us/step - loss: 0.3562 - accuracy: 0.9467 - val_loss: 0.3628 - val_accuracy: 0.9436
Epoch 286/500
350/350 [==============================] - 0s 1ms/step - loss: 0.3556 - accuracy: 0.9465 - val_loss: 0.3622 - val_accuracy: 0.9446
Epoch 287/500
350/350 [==============================] - 0s 883us/step - loss: 0.3550 - accuracy: 0.9464 - val_loss: 0.3

Epoch 337/500
350/350 [==============================] - 0s 934us/step - loss: 0.3278 - accuracy: 0.9571 - val_loss: 0.3347 - val_accuracy: 0.9536
Epoch 338/500
350/350 [==============================] - 0s 911us/step - loss: 0.3273 - accuracy: 0.9571 - val_loss: 0.3343 - val_accuracy: 0.9536
Epoch 339/500
350/350 [==============================] - 0s 946us/step - loss: 0.3268 - accuracy: 0.9574 - val_loss: 0.3338 - val_accuracy: 0.9532
Epoch 340/500
350/350 [==============================] - 0s 886us/step - loss: 0.3263 - accuracy: 0.9576 - val_loss: 0.3333 - val_accuracy: 0.9536
Epoch 341/500
350/350 [==============================] - 0s 931us/step - loss: 0.3259 - accuracy: 0.9580 - val_loss: 0.3328 - val_accuracy: 0.9536
Epoch 342/500
350/350 [==============================] - 0s 923us/step - loss: 0.3254 - accuracy: 0.9582 - val_loss: 0.3324 - val_accuracy: 0.9536
Epoch 343/500
350/350 [==============================] - 0s 914us/step - loss: 0.3249 - accuracy: 0.9586 - val_loss: 0

Epoch 393/500
350/350 [==============================] - 0s 851us/step - loss: 0.3034 - accuracy: 0.9643 - val_loss: 0.3107 - val_accuracy: 0.9636
Epoch 394/500
350/350 [==============================] - 0s 788us/step - loss: 0.3031 - accuracy: 0.9642 - val_loss: 0.3104 - val_accuracy: 0.9639
Epoch 395/500
350/350 [==============================] - 0s 830us/step - loss: 0.3027 - accuracy: 0.9643 - val_loss: 0.3100 - val_accuracy: 0.9636
Epoch 396/500
350/350 [==============================] - 0s 831us/step - loss: 0.3023 - accuracy: 0.9644 - val_loss: 0.3096 - val_accuracy: 0.9636
Epoch 397/500
350/350 [==============================] - 0s 804us/step - loss: 0.3019 - accuracy: 0.9646 - val_loss: 0.3092 - val_accuracy: 0.9639
Epoch 398/500
350/350 [==============================] - 0s 814us/step - loss: 0.3015 - accuracy: 0.9646 - val_loss: 0.3088 - val_accuracy: 0.9639
Epoch 399/500
350/350 [==============================] - 0s 785us/step - loss: 0.3011 - accuracy: 0.9646 - val_loss: 0

Epoch 449/500
350/350 [==============================] - 0s 791us/step - loss: 0.2837 - accuracy: 0.9712 - val_loss: 0.2913 - val_accuracy: 0.9704
Epoch 450/500
350/350 [==============================] - 0s 820us/step - loss: 0.2834 - accuracy: 0.9713 - val_loss: 0.2910 - val_accuracy: 0.9707
Epoch 451/500
350/350 [==============================] - 0s 834us/step - loss: 0.2831 - accuracy: 0.9715 - val_loss: 0.2906 - val_accuracy: 0.9707
Epoch 452/500
350/350 [==============================] - 0s 837us/step - loss: 0.2828 - accuracy: 0.9714 - val_loss: 0.2903 - val_accuracy: 0.9707
Epoch 453/500
350/350 [==============================] - 0s 817us/step - loss: 0.2824 - accuracy: 0.9715 - val_loss: 0.2900 - val_accuracy: 0.9707
Epoch 454/500
350/350 [==============================] - 0s 808us/step - loss: 0.2821 - accuracy: 0.9715 - val_loss: 0.2897 - val_accuracy: 0.9707
Epoch 455/500
350/350 [==============================] - 0s 797us/step - loss: 0.2818 - accuracy: 0.9715 - val_loss: 0

In [24]:
# 모델 평가
print(model.evaluate(x_data_test_norm, t_data_test_onehot)) # [0.2736673057079315, 0.9713333249092102]

188/188 [==============================] - 0s 690us/step - loss: 0.2737 - accuracy: 0.9713
[0.2736673057079315, 0.9713333249092102]


In [27]:
# 예측
predict_data = np.array([[187, 81]])
scaled_predict_data = scaler.transform(predict_data)
print(model.predict(scaled_predict_data)) # [[0.02638307 0.703346   0.2702709 ]]
                                          # 클래스별 확률이 출력된다. 가장 높은 확률의 클래스가 예측된 클래스이다.

1/1 [==============================] - 0s 15ms/step
[[0.02638307 0.703346   0.2702709 ]]
